# Overlap Check for Submission

The baseline submission failed with 'Overlapping trees in group 008'. Let's validate the submission properly.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

getcontext().prec = 30

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(0), float(tip_y)),
            (float(top_w / 2), float(tier_1_y)),
            (float(top_w / 4), float(tier_1_y)),
            (float(mid_w / 2), float(tier_2_y)),
            (float(mid_w / 4), float(tier_2_y)),
            (float(base_w / 2), float(base_y)),
            (float(trunk_w / 2), float(base_y)),
            (float(trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(base_y)),
            (float(-base_w / 2), float(base_y)),
            (float(-mid_w / 4), float(tier_2_y)),
            (float(-mid_w / 2), float(tier_2_y)),
            (float(-top_w / 4), float(tier_1_y)),
            (float(-top_w / 2), float(tier_1_y)),
        ])

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    subset = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in subset.iterrows():
        x = str(row['x']).lstrip('s')
        y = str(row['y']).lstrip('s')
        deg = str(row['deg']).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    return trees

def has_overlap(trees):
    if len(trees) <= 1:
        return False, None
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    intersection = poly.intersection(polygons[idx])
                    if intersection.area > 1e-12:
                        return True, (i, idx, intersection.area)
    return False, None

print('Validation functions defined')

In [ ]:
# Load the submission that was submitted
df = pd.read_csv('/home/submission/submission.csv')
print(f'Submission shape: {df.shape}')
print(df.head())

In [ ]:
# Check group 008 specifically
print('Checking group 008...')
trees = load_trees_for_n(df, 8)
print(f'Found {len(trees)} trees')
overlap, details = has_overlap(trees)
print(f'Has overlap: {overlap}')
if details:
    print(f'Overlap details: trees {details[0]} and {details[1]}, area={details[2]:.2e}')

In [ ]:
# Check all groups for overlaps
print('Checking all groups for overlaps...')
overlapping_groups = []
for n in range(1, 201):
    trees = load_trees_for_n(df, n)
    if len(trees) != n:
        print(f'Warning: N={n} has {len(trees)} trees')
        continue
    overlap, details = has_overlap(trees)
    if overlap:
        overlapping_groups.append((n, details))
        print(f'  N={n}: OVERLAP - trees {details[0]} and {details[1]}, area={details[2]:.2e}')

if not overlapping_groups:
    print('\nNo overlaps found!')
else:
    print(f'\nFound {len(overlapping_groups)} groups with overlaps')

In [ ]:
# Let's also check the original failed submission
original_path = '/home/nonroot/snapshots/santa-2025/21328309254/code/submission.csv'
df_orig = pd.read_csv(original_path)

print('Checking original submission group 008...')
trees = load_trees_for_n(df_orig, 8)
overlap, details = has_overlap(trees)
print(f'Has overlap: {overlap}')
if details:
    print(f'Overlap details: trees {details[0]} and {details[1]}, area={details[2]:.2e}')

In [ ]:
# Check the ensemble_best.csv that scored 27.4
ensemble_path = '/home/nonroot/snapshots/santa-2025/21322576451/code/experiments/ensemble_best.csv'
df_ensemble = pd.read_csv(ensemble_path)

print('Checking ensemble_best.csv for overlaps...')
overlapping_groups = []
for n in range(1, 201):
    trees = load_trees_for_n(df_ensemble, n)
    if len(trees) != n:
        continue
    overlap, details = has_overlap(trees)
    if overlap:
        overlapping_groups.append((n, details))
        if len(overlapping_groups) <= 10:
            print(f'  N={n}: OVERLAP - trees {details[0]} and {details[1]}, area={details[2]:.2e}')

if not overlapping_groups:
    print('\nNo overlaps found in ensemble_best.csv!')
else:
    print(f'\nFound {len(overlapping_groups)} groups with overlaps in ensemble_best.csv')